## Edgeに自動的にclickする

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

url = 'https://www.amazon.co.jp/'
options=webdriver.EdgeOptions()
options.add_argument('--headless')
driver = webdriver.Edge(options=options)
driver.get(url)

find = driver.find_elements(by='id', value='twotabsearchtextbox')
find[0].send_keys(input('ban muon mua sach gi'))
find[0].send_keys(Keys.RETURN )

## linkを取り使う

In [3]:

def get_url():
    page_source=BeautifulSoup(driver.page_source,'html.parser')
    profiles=page_source.find_all('a',class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    
    all_profiles=[]
    for profile in profiles:
        profile_id=profile.get('href')
        profile_url='https://www.amazon.co.jp'+profile_id
        if profile_url not in all_profiles:
            all_profiles.append(profile_url)
    return all_profiles



In [4]:
def scrape_data():
    d={'id':[],"name":[],'price':[],"rating":[],"comment":[]}
    for link in get_url():
        driver.get(link)
        d['id'].append(link)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        try:
            name = page_source.find('span', id='productTitle').text.strip()
            d['name'].append(name)
        except:
                 d['name'].append("")
        try:
            price = page_source.find('span', class_='a-price-whole').text.strip()
            d['price'].append(price)

        except:
             d['price'].append("")    
        try:
            rating = page_source.find('span', class_='a-icon-alt').text.strip()
            d['rating'].append(rating)
        except:
             d['rating'].append("")
        comment = page_source.find('span',id="acrCustomerReviewText")
        if comment is not None:
            d['comment'].append(comment.text.strip())
        else:
            d['comment'].append('')
        print('\n')
    
    return d


In [ ]:

df=pd.DataFrame.from_dict(scrape_data())
df['price'].replace("",np.nan,inplace=True)
df=df.dropna(subset=['price'])
df['price']=df['price'].str.replace(',',"").astype(int)


In [13]:
df['comment'].replace("",np.nan,inplace=True)
df=df.dropna(subset=['comment'])
df

C:\Users\MSI GL63\AppData\Local\Temp\ipykernel_24804\4151439314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'].replace("",np.nan,inplace=True)


,id,name,price,rating,comment,comment_count
4,https://www.amazon.co.jp/Lexicon-Love-ABC/dp/B...,Lexicon of Love,1547.0,5つ星のうち4.8,670個の評価,670
8,https://www.amazon.co.jp/%E3%81%AF%E3%81%9A%E3...,はずる キャスト ABC(エービーシー) 【難易度レベル1】,881.0,5つ星のうち4.1,"1,665個の評価","1,665"
10,https://www.amazon.co.jp/%E3%82%B5%E3%83%B3%E3...,サンスター文具 知育パズル A B C それいけ!アンパンマン 6440010A,660.0,5つ星のうち5.0,2個の評価,2
12,https://www.amazon.co.jp/%E3%80%90%E3%82%B5%E3...,【サンスター文具】 知育パズルABC それいけ!アンパンマン 5400018E,440.0,5つ星のうち4.5,"1,083個の評価","1,083"
13,https://www.amazon.co.jp/ABC-ABC%E3%82%B5%E3%8...,ABCサンバルアスリ チリソース 335ml×３本,1620.0,5つ星のうち4.5,193個の評価,193
16,https://www.amazon.co.jp/%E3%81%8F%E3%82%82%E3...,くもん出版 たっぷり7つの遊び! ABCブロック,1650.0,5つ星のうち4.1,10個の評価,10
17,https://www.amazon.co.jp/COUPPEY-%E3%83%94%E3%...,LE COUPPEY ピアノの練習ABC 安川加寿子 校訂・注,1320.0,5つ星のうち4.3,93個の評価,93
18,https://www.amazon.co.jp/%E3%83%A9%E3%83%BC%E3...,ラーニングリソーシズ パズルカード ABC 英語 幼児向け LER6085 正規品,2178.0,5つ星のうち4.7,"2,117個の評価","2,117"
20,https://www.amazon.co.jp/ABC-Letter-Tracing-Pr...,ABC Letter Tracing for Preschoolers: A Fun Boo...,875.0,5つ星のうち4.7,"6,919個の評価","6,919"
22,https://www.amazon.co.jp/%E3%82%A8%E3%83%AA%E3...,エリナ ABCクィック《450g(30g×15)》,5235.0,5つ星のうち3.7,13個の評価,13


## 最高価格と最低価格は何ですか？

In [7]:
valuse=df.groupby('name')['price'].sum()
valuse_sorted=valuse.sort_values(ascending=False)
max=valuse_sorted.iloc[0]
min=valuse_sorted.iloc[-1]
print("最高価格は '{}'という商品 {}円があります、\n逆に最低価格は ’{}’という商品 {}円があります".format(valuse_sorted.index[0],max,valuse_sorted.index[-1],min))


最高価格は '【メーカー特典3種付き・初回プレス特典付き・3形態セット】Mr.5【 初回盤A+初回盤B+通常盤(初回プレス) 】【通常盤 初回プレス封入特典:アナザージャケット（塗り絵Ver.)封入】【初回封入特典:期間限定動画A+B+C視聴シリアルナンバー封入】(特典:King & Princeオリジナルフォトカード(A6サイズ)付き)(特典:King & Princeオリジナルクリアポスター(A4サイズ)付き)(特典:King & Princeオリジナルアナザージャケット5種セット付き)'という商品 18580円があります、
逆に最低価格は ’【サンスター文具】 知育パズルABC それいけ!アンパンマン 5400018E’という商品 440円があります


## 評価（rating）が最も多い星数の商品数と最も少ない星数の商品数

In [8]:
df['rating'].str.slice(-3,).astype(float)
valuse_rating=df.groupby('name')['rating'].sum()
sorted_valuse_rating=valuse_rating.sort_values(ascending=False)
max_rating=sorted_valuse_rating.iloc[0]
min_rating=sorted_valuse_rating.iloc[-1]
print("評価が最も多い星数の商品数は{}というの商品が{}です、\n逆に最も少ない星数の商品数は{}という商品が{}です".format(sorted_valuse_rating.index[0],max_rating,sorted_valuse_rating.index[-1],min_rating))

評価が最も多い星数の商品数はサンスター文具 知育パズル A B C それいけ!アンパンマン 6440010Aというの商品が5つ星のうち5.0です、
逆に最も少ない星数の商品数はエリナ ABCクィック《450g(30g×15)》という商品が5つ星のうち3.7です


## 最も多くのレビューを持つ商品と最も少ないのレビューを持つ商品

In [9]:
df['comment_count']=df['comment'].str.replace("個の評価","")
valuse_commet=df.groupby('name')['comment_count'].sum()
sorte_valuse_commet=valuse_commet.sort_values(ascending=False)
max_commet=sorte_valuse_commet.iloc[0]
min_commet=sorte_valuse_commet.iloc[-1]
print('最も多くのレビューを持つ商品は{}という商品が{}個あります、\n最も少ないのレビューを持つ商品は{}という商品が{}個あります'.format(sorte_valuse_commet.index[0],max_commet,sorte_valuse_commet.index[-1],min_commet))

最も多くのレビューを持つ商品はABCのでんしゃじてん (せんろをなぞれるシリーズ)という商品が99個あります、
最も少ないのレビューを持つ商品は【サンスター文具】 知育パズルABC それいけ!アンパンマン 5400018Eという商品が1,083個あります


C:\Users\MSI GL63\AppData\Local\Temp\ipykernel_24804\1019177729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_count']=df['comment'].str.replace("個の評価","")


## 産品名が重複している製品

In [10]:
duplicate_count = len(df[df.duplicated(subset=['name'], keep=False)].value_counts())


In [11]:
unique_names = df['name'].unique()

for name in unique_names:
    count = df[df['name'] == name]['name'].count()
    if count > 1:
       print(f'産品名が重複している製品は{name}です。')



## 2000円以上で評価が最高の製品は何個ありますか

In [12]:

df['price'] = df['price'].astype(float)
top_products = df[(df['price'] > 2000) & (df["rating"] >"4.5")]
counts=len(top_products)
print(f"2000円以上で評価が最高の製品は{count}個あります")
print('それは：')
top_products[['name']]

2000円以上で評価が最高の製品は1個あります
それは：


C:\Users\MSI GL63\AppData\Local\Temp\ipykernel_24804\1302956413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].astype(float)


,name
18,ラーニングリソーシズ パズルカード ABC 英語 幼児向け LER6085 正規品
22,エリナ ABCクィック《450g(30g×15)》
25,あいうえお・ABC でんしゃタブレット (音の出る知育絵本)
28,【メーカー特典3種付き・初回プレス特典付き・3形態セット】Mr.5【 初回盤A+初回盤B+通...
40,ABC座 ジャニーズ伝説2017[DVD]
41,ABC HOBBY ベアリング リフレッシュセット レッド 69012
